In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.optimizers import SGD

Using TensorFlow backend.


In [4]:
#coding:utf-8
"""
Author:wepon
Source:https://github.com/wepe
file:data.py
"""

import os
from PIL import Image
import numpy as np

#读取文件夹mnist下的42000张图片，图片为灰度图，所以为1通道，
#如果是将彩色图作为输入,则将1替换为3，并且data[i,:,:,:] = arr改为data[i,:,:,:] = [arr[:,:,0],arr[:,:,1],arr[:,:,2]]
def load_data():
    data = np.empty((42000,1,28,28),dtype="float32") #42000个一通道的数组，数组格式为28*28
    label = np.empty((42000,),dtype="uint8")

    imgs = os.listdir("E:/machine_data/mnist")   #os.listdir获取目录的内容
    num = len(imgs)
    for i in range(num):
        img = Image.open("E:/machine_data/mnist/"+imgs[i])
        arr = np.asarray(img,dtype="float32")  #将输入数据（列表的列表，元组的元组，元组的列表等）转换为矩阵形式
        data[i,:,:,:] = arr
        label[i] = int(imgs[i].split('.')[0])
    return data,label

In [5]:
from keras.utils import np_utils, generic_utils

In [21]:
#加载数据
data, label = load_data()
print(data.shape, ' samples')

#label为0~9共10个类别，keras要求格式为binary class matrices,转化一下，直接调用keras提供的这个函数
label1 = np_utils.to_categorical(label, 10)
print (label1[1:3])

(42000, 1, 28, 28)  samples
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [22]:
X_train=data
Y_train=label1

In [23]:
X_train[:,0,:,:].shape

(42000, 28, 28)

In [24]:
X_train = X_train[:,0,:,:].reshape(42000, 784)

In [25]:
X_train.shape

(42000, 784)

In [ ]:

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 784-dimensional vectors.
model.add(Dense(512, input_shape=(784,)))   # 全连接层， 64个神经元
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))         # 最后一个全连接层用 softmax当激活函数

sgd = SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=False)  # 用随机梯度下降优化，nesterov？？？？？
model.compile(loss='categorical_crossentropy', optimizer=sgd ,metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(X_train, Y_train, nb_epoch=20, batch_size=200,shuffle=True,verbose=1)
#score = model.evaluate(X_test, Y_test, batch_size=16)

In [79]:
# 输出训练好的模型在测试集上的表现
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0899575967312
Test accuracy: 0.1135


In [34]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_22 (Dense)                 (None, 512)           401920      dense_input_8[0][0]              
____________________________________________________________________________________________________
activation_22 (Activation)       (None, 512)           0           dense_22[0][0]                   
____________________________________________________________________________________________________
dropout_15 (Dropout)             (None, 512)           0           activation_22[0][0]              
____________________________________________________________________________________________________
dense_23 (Dense)                 (None, 256)           131328      dropout_15[0][0]                 
___________________________________________________________________________________________

## 某博客中的例子

In [41]:
from __future__ import print_function
# 导入numpy库， numpy是一个常用的科学计算库，优化矩阵的运算
import numpy as np
np.random.seed(1337)

# 导入mnist数据库， mnist是常用的手写数字库
from keras.datasets import mnist
# 导入顺序模型
from keras.models import Sequential
# 导入全连接层Dense， 激活层Activation 以及 Dropout层
from keras.layers.core import Dense, Dropout, Activation
# 导入优化器RMSProp
from keras.optimizers import RMSprop
# 导入numpy工具，主要是用to_categorical来转换类别向量
from keras.utils import np_utils

In [42]:
# 设置batch的大小
batch_size = 128
# 设置类别的个数
nb_classes = 10
# 设置迭代的次数
nb_epoch = 20

In [31]:
# keras中的mnist数据集已经被划分成了60,000个训练集，10,000个测试集的形式，按以下格式调用即可
(X_train, y_train), (X_test, y_test) = mnist.load_data()

15269888/15296311 [============================>.] - ETA: 0s

In [33]:
print (X_train.shape)
print (y_train.shape)

(60000, 28, 28)
(60000,)


In [43]:
# X_train原本是一个60000*28*28的三维向量，将其转换为60000*784的二维向量
X_train = X_train.reshape(60000, 784)
# X_test原本是一个10000*28*28的三维向量，将其转换为10000*784的二维向量
X_test = X_test.reshape(10000, 784)
# 将X_train, X_test的数据格式转为float32存储
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# 归一化
X_train /= 255
X_test /= 255
# 打印出训练集和测试集的信息
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [44]:
'''
将类别向量(从0到nb_classes的整数向量)映射为二值类别矩阵，
相当于将向量用one-hot重新编码'''
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [45]:
# 建立顺序型模型
model = Sequential()
'''
模型需要知道输入数据的shape，
因此，Sequential的第一层需要接受一个关于输入数据shape的参数，
后面的各个层则可以自动推导出中间数据的shape，
因此不需要为每个层都指定这个参数
''' 

# 输入层有784个神经元
# 第一个隐层有512个神经元，激活函数为ReLu，Dropout比例为0.2
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# 第二个隐层有512个神经元，激活函数为ReLu，Dropout比例为0.2
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# 输出层有10个神经元，激活函数为SoftMax，得到分类结果
model.add(Dense(10))
model.add(Activation('softmax'))

# 输出模型的整体信息
# 总共参数数量为784*512+512 + 512*512+512 + 512*10+10 = 669706
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_46 (Dense)                 (None, 512)           401920      dense_input_16[0][0]             
____________________________________________________________________________________________________
activation_46 (Activation)       (None, 512)           0           dense_46[0][0]                   
____________________________________________________________________________________________________
dropout_31 (Dropout)             (None, 512)           0           activation_46[0][0]              
____________________________________________________________________________________________________
dense_47 (Dense)                 (None, 512)           262656      dropout_31[0][0]                 
___________________________________________________________________________________________

In [46]:
'''
配置模型的学习过程
compile接收三个参数：
1.优化器optimizer：参数可指定为已预定义的优化器名，如rmsprop、adagrad，
或一个Optimizer类对象，如此处的RMSprop()
2.损失函数loss：参数为模型试图最小化的目标函数，可为预定义的损失函数，
如categorical_crossentropy、mse，也可以为一个损失函数
3.指标列表：对于分类问题，一般将该列表设置为metrics=['accuracy']
'''
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

'''
训练模型
batch_size：指定梯度下降时每个batch包含的样本数
nb_epoch：训练的轮数，nb指number of
verbose：日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为epoch输出一行记录
validation_data：指定验证集
fit函数返回一个History的对象，其History.history属性记录了损失函数和其他指标的数值随epoch变化的情况，
如果有验证集的话，也包含了验证集的这些指标变化情况
'''
history = model.fit(X_train, Y_train,
                    batch_size = batch_size,
                    nb_epoch = nb_epoch,
                    verbose = 1,
                    validation_data = (X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 10s - loss: 0.8578 - acc: 0.7366 - val_loss: 0.4189 - val_acc: 0.8819
Epoch 2/20
60000/60000 [==============================] - 10s - loss: 0.3680 - acc: 0.8900 - val_loss: 0.2907 - val_acc: 0.9129
Epoch 3/20
60000/60000 [==============================] - 11s - loss: 0.2807 - acc: 0.9163 - val_loss: 0.2297 - val_acc: 0.9305
Epoch 4/20
60000/60000 [==============================] - 10s - loss: 0.2235 - acc: 0.9328 - val_loss: 0.1924 - val_acc: 0.9416
Epoch 5/20
60000/60000 [==============================] - 10s - loss: 0.1828 - acc: 0.9454 - val_loss: 0.1648 - val_acc: 0.9492
Epoch 6/20
60000/60000 [==============================] - 10s - loss: 0.1564 - acc: 0.9534 - val_loss: 0.1403 - val_acc: 0.9571
Epoch 7/20
60000/60000 [==============================] - 10s - loss: 0.1367 - acc: 0.9582 - val_loss: 0.1181 - val_acc: 0.9639
Epoch 8/20
60000/60000 [==============================

In [47]:
# 按batch计算在某些输入数据上模型的误差
score = model.evaluate(X_test, Y_test, verbose=0)
# 输出训练好的模型在测试集上的表现
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0768508187469
Test accuracy: 0.9781
